<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

QTL-seq入門 (2): リードアライメント、SNP検出
---

　前回に続き、次世代シーケンサーを使った遺伝子マッピング手法のひとつ、QTL-seqを勉強・体験していきます。

QTL-seq入門シリーズ: 
- [QTL-seq入門 (1): 参照配列（リファレンス配列）の準備](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2022/QTLseq_1.ipynb)
- QTL-seq入門 (2): リードアライメント、SNP検出　←今回
- [QTL-seq入門 (3): SNP-indexの算出、グラフ描画](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2022/QTLseq_3.ipynb)


<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_workflow.png?raw=true" alt="QTL-seq workflow" height="180px" align="middle">

In [ ]:
#########################
##  実習の前に実行してください。 ##
#########################
%%bash
## ソフトウェアのインストール
wget -q -O install.sh https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/scripts/install_for_alignment.sh
sh install.sh && rm -f install.sh
pip install -q igv-notebook==0.3.1
pip list | grep -e "igv-notebook"
## サンプルファイルのダウンロード
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/scripts/igv_prep.py -O igv_prep.py
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/genome.tar.gz -O - | tar -zxf - 2>/dev/null
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/reads.tar.gz -O - | tar -zxf - 2>/dev/null
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/bam.tar.gz -O - | tar -zxf - 2>/dev/null
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/ngs.tar.gz -O - | tar -zxf - 2>/dev/null

# 今回の勉強内容

1. リードアライメントとSNP検出を学ぶ
  1. 品種Aのリードアライメント
  1. 品種BのリードアライメントとSNP検出
  1. 交配後のF<small>1</small>個体のアライメントデータ
  1. 交配後のF<small>2</small>個体のアライメントデータ
1. バルクシーケンシングを学ぶ
  1. グループで分離していないSNPを探す
  1. QTL-seq: バルクシーケンシング
1. 実例: イネのQTL-seq
1. シーケンシングの種類と特徴


# 1. リードアライメントとSNP検出を学ぶ

　QTL-seqでは、参照配列（リファレンス配列）を用意したあと、次にシーケンシングデータ（リード配列）を参照配列に並べます。配列が似ている箇所を参照配列から探し、その場所にリードを並べます<sup>※1</sup>。その工程のことを**リードアライメント**（または、**リードマッピング**）と言います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/read_alignment.png?raw=true" alt="Read alignment" height="80px" align="middle">

<small>※1 リードアライメントの仕組みに興味がある方は以下のWebページを参照してください。<br>　[How Do We Compare Biological Sequences?](https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5) (Bioinformatics Algorithms / Phillip Compeau)</small>

　リードアライメントをおこなうと、参照配列とリード配列の間の塩基の違い、**SNP（Single Nucleotide Polymorphism; 一塩基多型）を検出**できます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/SNP_detection.png?raw=true" alt="SNP detection" height="160px" align="middle">

　ここでは、実際にリードアライメントをおこない、その結果を確認し、次世代シーケンスデータのSNP検出についてを学びましょう。



## 1-1. 品種Aのリードアライメント

　ここでは、以下2つの配列データを使います。
- 参照配列: 品種Aのゲノム配列（300bp）
  - /content/genome/CultivarA.fa
- リード配列: 品種Aのリード配列（合計約240本の**ペアエンドリード**<sup>※2</sup>、各20bp）
  - /content/reads/CultivarA_read1.fq
  - /content/reads/CultivarA_read2.fq

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/paired_end_read.png?raw=true" alt="DNA sequencing and paired-end read" height="230px" align="middle">

<small>※2 配列断片の両端を挟むようにシーケンシングして得られたリードのことを「ペアエンドリード (Paired-end read)」と言います。1組のペアエンドリードで2本分のリード配列データを得られます。一方からシーケンシングした配列がSAMPLE_read1.fqに入っており、もう一方からシーケンシングした配列がSAMPLE_read2.fqに入っています。2ファイルで1サンプル分のデータのため、ペアとして扱うのが一般的です。</small>


### 塩基配列データ形式: FASTAとFASTQ

　塩基配列をデータとして扱う際によく使う記述形式は**FASTA**です。配列名と配列本体をセットで記述しておきたい場合によく使われます（例えば、参照配列（ゲノム配列）など）。
```shell
# FASTA形式の見方
>塩基配列名
塩基配配列本体

# 例
>chr1
ATTGCTGGCGTTGGCGAGCTAAACAACCA
GTGCACGATGAGCGTCCTCTTTTTTAATA
GCGATCGGAT
>chr2
CTATTTGGTAGCTCCCGTC
>chr3
TACAACTTGGATTCGGGGTTTGGGTAATG
GGGATAAGACCTTGGACGCT
```

　次世代シーケンサーから塩基配列を出力する際、リードの配列名と配列本体の情報だけでなく、各塩基のクオリティ情報（塩基の信頼度）も得られます。**FASTQ**は、配列名、配列本体に加えて、塩基のクオリティ情報を記述できる様式です。1文字で1塩基分の[Phredクオリティスコア](https://github.com/CropEvol/lecture/blob/master/textbook_2021/L03_phred_score.md)を記述できます。例えば、下記の例のread1について、最初の塩基「G」のスコアは「/」で、その次の塩基「A」のスコアは「<」です。

```shell
# FASTQ形式の見方
@塩基配列名
塩基配列本体
+
塩基のクオリティ情報

#　例
@read1
GAGGGGGAGCGGAAGAGTGG
+
/<3GGGGGFGFGFEGGGFG?
@read2
TCTGCCTCTCCTCCCTCTCT
+
GGGGEGGGFFGGGFD?4C?A
@read3
AAATAAATCCTCCTTCCTAC
+
GGCFGFFFF?FDDFEG?;FC
```

　クオリティ情報は、SNP検出（参照配列と違う塩基を見つけた時、シーケンシングエラーかSNP変異かの判断）や検出したSNPの信頼度計算などに使われます。

### -------------------------------------

　今回、実際のゲノム解析でも良く使われている[**BWA (Burrows-Wheeler Alignment)**](https://bio-bwa.sourceforge.net/)と呼ばれるソフトウェアで、リードアライメントをおこないます。

　次のプログラムを実行して、リードアライメントをおこなってください。

In [ ]:
%%bash
## ------------------------------------------------------------------
## Alignment: CultivarA reads to CultivarA reference
## ------------------------------------------------------------------
## リードアライメントの事前準備
if [ ! -e genome/CultivarA.fa.bwt ]; then bwa index genome/CultivarA.fa; fi
## リードアライメント
bwa aln genome/CultivarA.fa reads/CultivarA_read1.fq > read1.sai 2>/dev/null
bwa aln genome/CultivarA.fa reads/CultivarA_read2.fq > read2.sai 2>/dev/null
bwa sampe genome/CultivarA.fa read1.sai read2.sai reads/CultivarA_read1.fq reads/CultivarA_read2.fq > CultivarA.sam
samtools sort -O bam CultivarA.sam > CultivarA.bam
samtools index CultivarA.bam
rm -fR read1.sai read2.sai

　アライメント後、次のプログラムを実行して、アライメント結果を見てみましょう。

<small>※ リードアライメント結果を見るために、実際のゲノム解析でも良く使われているソフトウェア [**IGV (Integrative Genomics Viewer)**](https://software.broadinstitute.org/software/igv/home) のPython版を使います。</small>

In [ ]:
## アライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
A = BamTrack({ "name":"CultivarA", "path":"CultivarA.bam", "indexPath":"CultivarA.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
b.load_track(A)

　次世代シーケンシングリードは、多量に得られる代わりに、シーケンシングエラーを少なからず含んでいます。十分なリード量があれば、それがエラーによる塩基なのか、変異塩基なのかを判別できます。

　品種Aのリードは、品種Aのゲノム配列から由来しているので、シーケンシングエラー以外は参照配列と同じ配列を持っています。

## 1-2. 品種BのリードアライメントとSNP検出

　同様に、次の二つのプログラムを順番に実行して、品種Bのリードアライメントをおこない、その結果を確認してください。

使用するデータ: 
- 参照配列: 品種Aのゲノム配列（300bp）
  - /content/genome/CultivarA.fa
- リード配列: 品種Bのリード配列（合計約240本のペアエンドリード、各20bp）
  - /content/reads/CultivarB_read1.fq
  - /content/reads/CultivarB_read2.fq

In [ ]:
%%bash
## ------------------------------------------------------------------
## Alignment: CultivarB reads to CultivarA reference
## ------------------------------------------------------------------
## リードアライメントの事前準備
if [ ! -e genome/CultivarA.fa.bwt ]; then bwa index genome/CultivarA.fa; fi
## リードアライメント
bwa aln genome/CultivarA.fa reads/CultivarB_read1.fq > read1.sai 2>/dev/null
bwa aln genome/CultivarA.fa reads/CultivarB_read2.fq > read2.sai 2>/dev/null
bwa sampe genome/CultivarA.fa read1.sai read2.sai reads/CultivarB_read1.fq reads/CultivarB_read2.fq > CultivarB.sam
samtools sort -O bam CultivarB.sam > CultivarB.bam
samtools index CultivarB.bam
rm -fR read1.sai read2.sai

In [ ]:
## アライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
B = BamTrack({ "name":"CultivarB", "path":"CultivarB.bam", "indexPath":"CultivarB.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
b.load_track(B)

　リードアライメント結果では、参照配列（品種Aのゲノム配列）とリード配列（品種Bの配列）の間で異なった塩基の場所がカラーで示されています。一部の塩基箇所について見てみると、ほぼすべてのリードが異なる塩基をもっており、エラー由来の塩基ではないことがわかります。したがって、この箇所が品種Aと品種Bの間の**SNP**と考えられます。

## 1-3. 交配後のF<small>1</small>個体のアライメントデータ

　品種Aと品種Bを人為的に交配して作った個体（F<small>1</small>個体）は、品種Aと品種Bから一つずつ配偶子を受け取って生じた個体であるため、品種Aと品種BのそれぞれのSNPをもっています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/f1_hybrid.png?raw=true" alt="SNP allele heterozygous of F1" height="200px" align="middle">

　F<small>1</small>個体ではどのようなリードアライメント結果が得られるか見てみましょう。以下を実行すると、F<small>1</small>個体のリードアライメント結果を表示できます。

<small>※ ここからは、bwaによるリードアライメントを省略し、事前に用意したアライメント結果を見ていきます。</small>

In [ ]:
## アライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
B = BamTrack({ "name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai", "viewAsPairs":True, "displayMode":"SQUISHED", "height":150 })
F1= BamTrack({ "name":"F1",       "path":"bam/F1.bam",       "indexPath":"bam/F1.bam.bai",       "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
b.load_track(B)
b.load_track(F1)

　各SNP箇所の塩基構成を見ると、多少ばらついているものの、二つのSNP塩基（以下、**SNPアリル**）がおよそ50%ずつ見つかります。

## 1-4. 交配後のF<small>2</small>世代のアライメントデータ

　F<small>1</small>個体を自殖して得た次の世代（F<small>2</small>世代）では、SNPアリルは各F<small>2</small>個体にランダムに遺伝されます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/SNP_allele_segregation.png?raw=true" alt="SNP allele segregation in F2" height="230px" align="middle">

　F<small>2</small>世代では各個体のリードアライメントがどのようになるか見てみましょう。次のプログラムを実行すると、F<small>2</small>世代の100個体のうち、指定のF<small>2</small>個体（例えば、F2_001からF2_005）のアライメント結果を見れます。

In [ ]:
## 表示するF2番号: 1~100までの数字を自由に組み合わせ可能
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2 = [1,2,3,4,5]

## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
Info = [BamTrack({ **{"name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"F1",       "path":"bam/F1.bam", "indexPath":"bam/F1.bam.bai"}, **bam_common_info })]
for f in sorted(list(set(F2))):
    Info += [ BamTrack({ **{"name":"F2_{:03}".format(f), 
                          "path":"/content/bam/F2_{:03}.bam".format(f),
                          "indexPath":"/content/bam/F2_{:03}.bam.bai".format(f)}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)

　個体ごとに各SNP箇所のSNPアリルの構成が異なり、F<small>2</small>世代でのSNPアリルの分離を確認できました。

　F<small>2</small>個体集団のように、SNPアリルがその場所ごとに独立に分離した（染色体がシャッフルされた）集団は、両親間の形質の違いを生み出す遺伝子（原因遺伝子）の位置を探索するのに役立ちます。

　次は、形質情報を考慮したリードアライメント結果を見ていきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/chromosome_shuffling.png?raw=true" alt="Chromosome shuffling" height="230px" align="middle">




# 2. バルクシーケンシングを学ぶ

　各個体の形質データは次のとおりです。
- F<small>2</small>各個体の形質: [形質データ表](https://github.com/CropEvol/lecture/blob/master/textbook_2022/QTLseq_2.pdf)
- 品種Aの形質: Short
- 品種Bの形質: Long
- F<small>1</small>個体の形質: Medium


## 2-1. グループ内で分離していないSNPを探す

　品種Aと同じ「Short」形質をもつF<small>2</small>個体（例えば、個体番号 `5, 7, 10, 15`）を4つ程度選び、それらのアライメント結果を表示してください。さらに、そのグループ内で分離していないSNP（固定SNP）を探してみてください。


In [ ]:
## 品種Aと同じShort形質をもつF2個体番号を半角数字で入力してください。
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2_S_bulk = []

## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
Info = [BamTrack({ **{"name":"CultivarA", "path":"bam/CultivarA.bam", "indexPath":"bam/CultivarA.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai"}, **bam_common_info }),]
for f in sorted(list(set(F2_S_bulk))):
    Info += [ BamTrack({ **{"name":"F2_{:03}".format(f), 
                          "path":"bam/F2_{:03}.bam".format(f),
                          "indexPath":"bam/F2_{:03}.bam.bai".format(f)}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)

　同様に、品種Bと同じ「Long」形質のF<small>2</small>個体についても、4個体分程度のアライメント結果を表示し、固定SNPを探してください。

例） 個体番号 `4, 8, 11, 21`

In [ ]:
## 品種Bと同じLong形質をもつF2個体番号を半角数字で入力してください。
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2_L_bulk = []

## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
Info = [BamTrack({ **{"name":"CultivarA", "path":"bam/CultivarA.bam", "indexPath":"bam/CultivarA.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai"}, **bam_common_info }),]
for f in sorted(list(set(F2_L_bulk))):
    Info += [ BamTrack({ **{"name":"F2_{:03}".format(f), 
                          "path":"bam/F2_{:03}.bam".format(f),
                          "indexPath":"bam/F2_{:03}.bam.bai".format(f)}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)

　形質ごとに4個体程度を表示し、各SNP箇所のSNPアリル構成を見ていくと、前から55番目の塩基に固定SNPを発見できます。

55番目の塩基:
- Short形質の品種Aでは「A」
- Short形質のF<small>2</small>個体でも「A」のみ
- Long形質の品種Bでは「T」
- Long形質のF<small>2</small>個体でも「T」のみ

　形質とSNPアリルに連関（Association）がみられることから、形質の違いを作っている原因変異かもしれません。

<small>※ 「Medium」形質のF<small>2</small>個体のアライメント結果には、55番塩基に2つのSNPアリルが見つかるはずです。余裕があれば確認してみてください。</small>


## 2-2. QTL-seq: バルクシーケンシング

　先ほどは、各個体のアライメント結果をサンプルごとに表示しました。**バルクシーケンシング**のリードは、いくつかの個体のリードをランダムにいくつか集めてプーリングし、"1サンプル分"のデータとして扱ったものとほぼ同じです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/read_sampling.png?raw=true" alt="Bulked DNA sequencing" height="250px" align="middle">

　今度は、バルクシーケンシングの結果を見てみましょう。以下の二つのコードセルを実行すると、任意の複数個体から集めたバルクシーケンシングリードのアライメント結果を表示できます。

　`F2_S_bulk`と`F2_L_bulk`の右辺の括弧`[]`内に、Short形質グループ（S-bulk）とLong形質グループ（L-bulk）に含める個体の番号を入力し、バルクシーケンシングリードのアライメント結果を確認してください。

In [ ]:
## プーリングするF2個体の番号
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2_S_bulk = [5, 7, 10, 15]
F2_L_bulk = [4, 8, 11, 21]

## Pooling
import re
def Pooling(Ns, bam_dir, samplingRate=-1, output="bulk.bam", verbose=False, samtools_path="samtools"):
    f2_list = [bam_dir+"/F2_{:03}.bam".format(f) for f in sorted(list(set(Ns)))]
    if samplingRate>=1.0:
        samplingRate = 1.0
    elif samplingRate<=0.0:
        samplingRate = float(1.0 / len(bam_dir) * 5.0)
    f2_str = " ".join(f2_list)
    !$samtools_path merge -o - $f2_str | samtools view -b -s $samplingRate > $output
    !$samtools_path index $output
    if verbose:
      f2_samples = [re.search(r".*/(.+?)$", i).group(1) for i in f2_list]
      print("Output: ", output)
      print("- Reads sampled from: "+",".join(f2_samples))
      print("- Sampling rate: %.3f" % samplingRate)
      
Pooling(F2_S_bulk, bam_dir="bam/", output="S_bulk.bam", verbose=True)
Pooling(F2_L_bulk, bam_dir="bam/", output="L_bulk.bam", verbose=True)

In [ ]:
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":200 }
S = BamTrack({ **{"name":"S-bulk", "path":"S_bulk.bam", "indexPath":"S_bulk.bam.bai"}, **bam_common_info })
L = BamTrack({ **{"name":"L-bulk", "path":"L_bulk.bam", "indexPath":"L_bulk.bam.bai"}, **bam_common_info })
## IGV表示
igv_notebook.init()
b = igv_notebook.Browser(ref)
b.load_track(S)
b.load_track(L)

### 実際のQTL-seqのバルクシーケンシングリード

　今回、個別のシーケンシングリードをいくつか集めてバルクシーケンシングリードを用意しました。しかし、実際のQTL-seqのバルクシーケンシングリードは少し異なります。

　QTL-seqでは。DNAの段階で形質ごとにDNAをプーリングしてシーケンシングします。そのため、プーリングしたどれかの個体のDNAから由来したリードであるのは間違いないですが、どの個体由来のリードかの特定はできません。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/bulk_sequencing.png?raw=true" alt="bulk sequencing" height="280px" align="middle">



　バルクDNAシーケンシングをおこなう利点は、分離集団（例えば、F<small>2</small>個体集団）の各個体を個別にシーケンシングして情報を得るより、かなりに費用が抑えられるためです。

例）シーケンシング費用が1サンプル10,000円の場合:  
- F<small>2</small>集団100個体の個別シーケンシング: 1,000,000円
- F<small>2</small>集団の二つの形質のバルクシーケンシング: 20,000円<sup>※3</sup>

<small>※3 QTL-seqでは形質ごとに数サンプル〜数十サンプル分に相当するデータ量が必要になるため、実際にはもう少しかかります。</small>

# 3. 実例: イネのQTL-seq

　ここでは、イネでQTL-seqをおこなった際の実データ（アライメントデータ）を見てみましょう。

　イネいもち病にかかりにくい品種（抵抗性品種）とかかりやすい品種（罹病性品種）の交配からできた分離集団で、抵抗性個体グループと罹病性個体グループのバルクシーケンシングし、罹病性品種のゲノム配列にアライメントデータです。

- 参照配列: 罹病性品種のゲノム配列
- バルクDNAシーケンス:
  - 抵抗性個体のバルクシーケンシングリード
  - 罹病性個体のバルクシーケンシングリード

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_takagi_etal_2013_2.png?raw=true" alt="Rice QTL-seq example (Takagi et al., 2013)" height="250px" align="middle">





　次の2つのコードセルを実行すると、イネのQTL-seqのアライメント結果（イネ6番染色体の15Mbのみ）を見れます。アライメント結果から、遺伝子がありそうなゲノム領域を探してみてください。

<small>※ [IGV-Web 版](https://igv.org/app/?sessionURL=blob:zZJLb8IwDID_i89dHwy2tTck2GmgaQ9pEpoqt3VpRNq0TgqbEP99HrAT7MakSckljv3ZX7KFNbFVpoEEBn507Y_AA1uZzTPWraY51mQhKVFb8oCpJKYmJ0i2UKJ1.Pr0IImVc61NgoBx4y.Vq_qst8S5aRw1zs9NHXQdtTd3oyKw.7JpgQ4tuaBG1QSdk.pdwCqnNK_4Jo3SaDTL_BKlF9UU9PHHGFkKdh5ok_cyLXxHk.gqCr1wEEsPbBQki3cPHGO.khuLLbjPVjSAVOz3RjwwXBBDchWH4W0Ux4PR8HYYxnG087bQs77YAM9p1uuVf7jhZ1gLu5F3EsC50MUFnkBkKwHJ9GpNxXzfiuNePkxpuEYn5EMrR2eo1bKpBSlHqDVpumfqXiom.Xi6gCT0B1JO2Vbj58wU3znTt8fxfDKdSMpa0WZsH1Gx_eEc1Q8vrfrpd9XnQhdXfQL5R6rfd18-) でも同じデータを公開しています。ただし、一部ブラウザ（Firefoxなど）ではエラーが発生して見れないかもしれません。</small>



In [ ]:
## Colab上にデータファイルのダウンロードする
!pip install -q -U igv-notebook==0.3.1
!wget -q -O igv_prep.py    https://github.com/CropEvol/lecture/raw/master/textbook_2022/scripts/igv_prep.py
!wget -q -O rice_chr6_1_15Mb.fa    https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/rice_chr6_1_15Mb.fa
!wget -q -O rice_chr6_1_15Mb.fa.fai https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/rice_chr6_1_15Mb.fa.fai
!wget -q -O rice_R_bulk.bam    https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/R_bulk.sample.bam
!wget -q -O rice_R_bulk.bam.bai https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/R_bulk.sample.bam.bai
!wget -q -O rice_S_bulk.bam    https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/S_bulk.sample.bam
!wget -q -O rice_S_bulk.bam.bai https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/S_bulk.sample.bam.bai

In [ ]:
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
bam_common_info = { "displayMode":"SQUISHED", "alleleFreqThreshold":0.4, "height":200 }

## 準備: Reference info
ref    = RefTrack({ "fastaPath":"rice_chr6_1_15Mb.fa", "indexPath":"rice_chr6_1_15Mb.fa.fai", "id":"Rice" }, locus="chr6:1-10,000")
## 準備: Reads info
r_bulk = BamTrack({ **{"name":"R_bulk", "path":"rice_R_bulk.bam", "indexPath":"rice_R_bulk.bam.bai"}, **bam_common_info })
s_bulk = BamTrack({ **{"name":"S_bulk", "path":"rice_S_bulk.bam", "indexPath":"rice_S_bulk.bam.bai"}, **bam_common_info })
## IGV表示
igv_notebook.init()
b = igv_notebook.Browser(ref)
b.load_track(r_bulk)
b.load_track(s_bulk)

　イネの染色体は12本あります。表示したデータは、イネ6番染色体の一部領域のみ（chr6:1-15,000,000）です。この領域のアライメント結果だけでも多量のSNPがあり、情報量も多く、目視で原因遺伝子の場所（QTL）を探すのは大変です。

　次回、SNP情報をより理解しやすい形に変換し、QTL探索をおこないましょう。

# 4. シーケンシングの種類と特徴

　ここまでIGVと呼ばれるソフトウェアを利用して、リードのアライメント結果を見てきました。

　ここでは、次の4つのタイプのシーケンシング方法から得られたリードのアライメント結果を見てみましょう。
- 全ゲノムシーケンシング（Whole genome sequencing: WGS）
- RNAシーケンシング（RNA sequencing: RNA-seq）
- RADシーケンシング（RAD sequencing: RAD-seq）
- ロングリードシーケンシング（Long-read sequencing）


In [ ]:
## 準備: ライブラリなど
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
igv_notebook.init()
base = "/content/ngs/"
bam_common_info = { "showCoverage":False, "displayMode":"SQUISHED", "viewAsPairs":True, "height":100 }

## 準備: Reference info
ref = RefTrack({ "fastaPath":base+"Ref.genome.fa", "indexPath":base+"Ref.genome.fa.fai", "id":"Reference" }, locus="chr01:30,000-40,000")
genes = AnnotationTrack({ "name":"Genes", "path":base+"Ref.transcript.gff", "format":"gff", "displayMode":"SQUISHED" })
restsite= AnnotationTrack({ "name":"Restriction site", "path":base+"Ref.radseq_enzymes.bed", "format":"bed", "color":"#ff7f0e"})
## 準備: Reads info
Info = [BamTrack({ **{"name":"WGS", "path":base+"wgs.bam", "indexPath":base+"wgs.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"RNA-seq", "path":base+"rnaseq.bam", "indexPath":base+"rnaseq.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"RAD-seq", "path":base+"radseq.bam", "indexPath":base+"radseq.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"Long-read", "path":base+"pacbio.bam", "indexPath":base+"pacbio.bam.bai"}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)
b.load_track(genes)
b.load_track(restsite)

　それぞれのシーケンシング対象と得られるSNP、特徴をまとめると次のとおりです。

| 種類 | シーケンシング対象領域 / SNP探索領域 | 特徴 |
|:---|:---|:---|
| Whole genome sequencing | ゲノム全配列 　| 解析には多くのリードを必要とする。 |
| RNA sequencing | 発現遺伝子（mRNA）配列<br>（イントロンは含まない）　| 得られるリード量は遺伝子発現量に依存する。<br>必要なリード量もWGSに比べて少なく済む。 |
| RAD sequencing | 任意の制限酵素切断部位近くの配列　| 少ないリード量で解析をおこなえる。<br>類似シーケンシング手法: GBS, GRAS-Di, MIG-seq |
| Long-reads sequencing | ゲノム全配列　| 長いリードを得られる一方で、シーケンシングエラーを多く含む。<sup>※4</sup><br>ロングリード単独解析をおこなうには多くのリードが必要である。<br><br><small>※4 「HiFiリード」と呼ばれるエラーが少ないロングリードもある。</small> |


# まとめ

　今回、QTL-seqのリードアライメントとSNP検出、バルクシーケンシングについて勉強しました。また、シーケンシングの種類とその特徴についても勉強しました。
- リードアライメント
  - 似た配列の場所を参照配列から探し、参照配列にリードを並べる
- SNP検出
  - 参照配列とリード配列を比較し、信頼性のある塩基の違いSNPを探す
- バルクシーケンシング
  - 複数個体のDNAをプーリングし、そのバルクDNAをシーケンシングする
  - QTL-seqでは、形質ごとにDNAをプーリングし、シーケンシングする
- シーケンシングの種類と特徴
  - 全ゲノムシーケンシング: ゲノム全配列が対象。解析には多くのリードを必要とする
  - RNAシーケンシング: 発現遺伝子（mRNA）配列が対象。遺伝子発現量の影響を受ける
  - RADシーケンシング: 制限酵素切断部位近くの配列が対象。低コスト
  - ロングリードシーケンシング: ゲノム全配列。リードは長いがエラーを多く含む
  


<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>